In [16]:
# %apt install subversion
# !svn checkout https://github.com/theislab/chemCPA.git
%pip install rdkit
%pip install pyarrow
%pip install umap-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3564 sha256=ed36d8fb1da2ebf088c0add38a3a30b73cfa31f69a58423ad2de850c9d43f13c
  Stored in directory: /Users/samfields/Library/Caches/pip/wheels/72/1e/42/a9322736ec046a637487005e6b84c94617d9ac0bd

In [22]:
%pip install umap-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 17.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 48.1 MB/s eta 0:00:0000:0100:01
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86854 sha256=468e11fb0d1a419bc68411e6353f32333992f771441f9dd50fe93808385f62ee
  Stored in directory: /Users/samfields/Library/Caches/pip/wheels/0f/01/17/26951217a11fb724b7027c6dd5b620b6d368104f7e6d4171fc
Successfully built umap-learn

[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packag

In [43]:
import pandas as pd
import sklearn
import numpy as np
from tqdm import tqdm


from sklearn.model_selection import train_test_split
from rdkit.Chem import AllChem
from rdkit import Chem
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.preprocessing import MinMaxScaler


# Template Python Notebook
This notebook is meant to be an aggregate of all the code we wish to submit. No experimental code should be included in this notebook. Rather, you should duplicate this notebook when wisshing to develope new code, and after aggreement on the code and analysis the new code should then be copied into this notebook

Competition Link: https://www.kaggle.com/competitions/open-problems-single-cell-perturbations/rules

Single-Cell Perturbation Modeling Resource: https://www.sc-best-practices.org/conditions/perturbation_modeling.html 

## Data Paths

In [3]:
PATH='open-problems-single-cell-perturbations'


adata_obs_meta_path = PATH + '/adata_obs_meta.csv'
adata_train_path = PATH + '/adata_train.parquet'
de_train_path = PATH + '/de_train.parquet'
id_map_path = PATH + '/id_map.csv'
de_train_path = PATH + '/de_train.parquet'
multiome_obs_meta_path = PATH + '/multiome_obs_meta.csv'
multiome_train_path = PATH + '/multiome_train.parquet'
multiome_var_meta_path = PATH + '/multiome_var_meta.csv'
excluded_ids_path = PATH + '/adata_excluded_ids.csv'

## Data Preprocessing and Splitting

**`de_train.parquet`** - Aggregated differential expression data in dense array format.

- `genes` A1BG, A1BG-AS1, …, ZZEF1 (numbering 18,211 in all) - Differential expression value (-log10(p-value) * sign(LFC)) for each gene. Here, LFC is the estimated log-fold change in expression between the treatment and control condition after shrinkage as calculated by Limma. Positive LFC means the gene goes up in the treatment condition relative to the control.
- `cell_type` - The annotated cell type of each cell based on RNA expression.
- `sm_name` - The primary name for the (parent) compound (in a standardized representation) as chosen by LINCS. This is provided to map the data in this experiment to the LINCS Connectivity Map data.
- `sm_lincs_id` - The global LINCS ID (parent) compound (in a standardized representation). This is provided to map the data in this experiment to the LINCS Connectivity Map data.
- `SMILES` - Simplified molecular-input line-entry system (SMILES) representations of the compounds used in the experiment. This is a 1D representation of molecular structure. These SMILES are provided by Cellarity based on the specific compounds ordered for this experiment.
- `control` - Boolean indicating whether this instance was used as a control.

**`adata_train.parquet `**- Unaggregated count and normalized data in COO sparse-array format. A supplement to de_train. In addition to the fields in de_train, this data also has:

- `obs_id` - This is a unique identifier assigned to each cell in the raw dataset.
- `gene` - Corresponds to the columns of de_train.
- `count` - The raw molecular counts for the gene expression data measured in the experiment as output by 10x CellRanger.
- `normalized_count` - These counts have been library size normalized and log(X+1) transformed.

**`adata_obs_meta.csv`** - Observation metadata for adata_train.

- `library_id` - A unique identifier for each library, which is a measurement made on pooled samples from each row of the plate. All cells from wells on the same row of the same plate will share a library_id.
-` plate_name` - A unique ID for all samples from the same plate.
well - The well location of the sample on each plate (this is standard across 96 well plate experiments). It is a concatenation of row and col.
row - Which row on the plate the sample came from.
- `col `- Which column on the plate the sample came from.
donor_id - Identifies the donor source of the sample, one of three.
- `cell_type` - The annotated cell type of each cell based on RNA expression. This matches the cell_type in the de_train.parquet.
- `cell_id` - This is included for consistency with LINCS Connectivity Map metadata, which denotes a cell_id for each cell line.
- `sm_name` - The primary name for the (parent) compound (in a standardized representation) as chosen by LINCS. This is provided to map the data in this experiment to the LINCS Connectivity Map data.
- `sm_lincs_id` - The global LINCS ID (parent) compound (in a standardized representation). This is provided to map the data in this experiment to the LINCS Connectivity Map data.
- `SMILES `- Simplified molecular-input line-entry system (SMILES) representations of the compounds used in the experiment. This is a 1D representation of molecular structure. These SMILES are provided by Cellarity based on the specific compounds ordered for this experiment.
- `dose_uM` - Dose of the compound in on a micro-molar scale. This maps to the pert_idose field in LINCS.
- timepoint_hr - Duration of treatment in hours. This maps to the pert_itime field in LINCS.
- `control` - Whether this observation was used as a control, True or False.

**`id_map.csv`** - Identifies the cell_type / sm_name pair to be predicted for the given id.

In [4]:
adata_train = pd.read_parquet(adata_train_path)
multiome_train = pd.read_parquet(multiome_train_path)

de_train = pd.read_parquet(de_train_path)

adata_excluded_ids = pd.read_csv(excluded_ids_path)

# Get gene list and convert to list
genes_to_exclude = adata_excluded_ids['gene'].tolist()
genes_to_exclude_set = set(genes_to_exclude)

#Find the intersection with de_train columns and drop the columns from de_train
genes_to_drop = genes_to_exclude_set & set(de_train.columns)
de_train_filtered = de_train.drop(columns=list(genes_to_drop))

adata_obs_meta =  pd.read_csv(adata_obs_meta_path)

id_map = pd.read_csv(id_map_path)

multiome_obs_meta = pd.read_csv(multiome_obs_meta_path)
multiome_var_meta = pd.read_csv(multiome_var_meta_path)

## Single Cell Aggregation

In [5]:
genes = de_train.iloc[:, 5:].columns.values.tolist() # Get all relevant genes
multiome_train = multiome_train[multiome_train['location'].isin(genes)] # keep relevant genes in multiome

multiome_train_ct = pd.merge(multiome_train, multiome_obs_meta, on='obs_id', how='left')
multiome_train_ct_meta = pd.merge(multiome_train_ct, multiome_var_meta, left_on='location', \
                                    right_on='location', how='left')
multiome_train_ct_meta_agg = multiome_train_ct_meta.groupby(['cell_type', 'location'])['normalized_count']\
    .agg(['mean', 'var']).reset_index()

In [6]:
transformed = []
cell_types = multiome_train_ct_meta_agg.cell_type.unique()

for cell_type in tqdm(cell_types):
    row = {'cell_type': cell_type}
    for gene in genes:
        temp = multiome_train_ct_meta_agg[(multiome_train_ct_meta_agg['cell_type']==cell_type) & \
                                  (multiome_train_ct_meta_agg['location']==gene)]
        mean = temp['mean'].values[0] if len(temp)>0 else 0
        var = temp['var'].values[0] if len(temp)>0 else 0
        row[f'{gene}_mean'] = mean
        row[f'{gene}_var'] = var
    transformed.append(row)

100%|█████████████████████████████████████████████| 6/6 [09:18<00:00, 93.05s/it]


In [69]:
multiome_train_ct_meta_agg_transformed = pd.DataFrame(transformed).fillna(0)

### Dimension Feature Reduction

In [70]:
# Get all columns except the first one
columns_except_first = multiome_train_ct_meta_agg_transformed.columns[1:]

# Store the first column separately
first_column = multiome_train_ct_meta_agg_transformed[multiome_train_ct_meta_agg_transformed.columns[0]]

#### TSNE

In [71]:
tsne = TSNE(n_components=6, perplexity=5, method='exact', random_state=42)
reduced = tsne.fit_transform(multiome_train_ct_meta_agg_transformed[columns_except_first])
reduced = pd.DataFrame(reduced, columns=[f'SC Component {x}' for x in range(len(reduced[0]))])
sc_pca = pd.concat([first_column, reduced], axis=1)


#### PCA

In [72]:
pca = PCA(n_components=6)  # You can choose the number of components you want to keep
reduced = pca.fit_transform(multiome_train_ct_meta_agg_transformed[columns_except_first])
reduced = pd.DataFrame(reduced, columns=[f'SC Component {x}' for x in range(len(reduced[0]))])
sc_pca = pd.concat([first_column, reduced], axis=1)


#### UMAP

In [90]:
from umap import umap_ as UMAP
# Create a UMAP object with the desired number of components
umap_model = UMAP.UMAP(n_components=200, init='random')

reduced = umap_model.fit_transform(multiome_train_ct_meta_agg_transformed[columns_except_first])
reduced = pd.DataFrame(reduced, columns=[f'SC Component {x}' for x in range(len(reduced[0]))])
sc_pca = pd.concat([first_column, reduced], axis=1)

/Users/samfields/Library/Python/3.9/lib/python/site-packages/umap/umap_.py:2433: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


### Compund Structure Embedding

In [91]:
mf_components = 200
# Get SMILES embeddings
de_train_filtered['Morgan_Fingerprints'] = de_train['SMILES'].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x), 2, nBits=1024))

de_train_filtered['Morgan_Fingerprints'] = de_train_filtered['SMILES'].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x), 2, nBits=1024))
# Perform PCA on SMILES embeddings
pca = PCA(n_components=mf_components, random_state=42)
features_pca = pca.fit_transform(list(de_train_filtered['Morgan_Fingerprints']))
features_pca = pd.DataFrame(features_pca, columns=[f'MF PCA {x}' for x in range(len(features_pca[0]))])


### Add Engineered Features

In [92]:
# Merge MF components
data = pd.merge(de_train_filtered, features_pca, left_index=True, right_index=True)

# merge sc components
data = pd.merge(data, sc_pca, on='cell_type', how='left')

### Normalize Engineered Features

In [93]:
# # Initialize the MinMaxScaler
# scaler = MinMaxScaler()

# norm_cols = features_pca.columns.values.tolist()
# norm_cols += sc_pca.columns.values.tolist()[1:]

# # Normalize all columns in the DataFrame
# data[norm_cols] = scaler.fit_transform(data[norm_cols])

## Creating Training / Testing Data

In [94]:
# creating test/train

# Identify the cell types for training and testing
training_cell_types = ['T cells (CD4+)', 'T cells (CD8+)', 'T cells (regulatory)', 'NK cells']
testing_cell_types = ['Myeloid cells', 'B cells']

# Filter data for training and testing
train_data = data[data['cell_type'].isin(training_cell_types)]
test_data = data[data['cell_type'].isin(testing_cell_types)]

# Randomly select 50% of samples from Myeloid and B cells for testing
test_samples = test_data.sample(frac=0.5, random_state=421)

# The remaining samples are used for training
train_samples = test_data.drop(test_samples.index)
train_samples = pd.concat([train_data, train_samples])

# declare features
xfeatures = list(features_pca.columns) + list(reduced.columns) # + list(cell_type_ohe.columns)
_yfeatures = ['cell_type', 'sm_name', 'sm_lincs_id', 'SMILES', 'control', 'Morgan_Fingerprints'] \
    + list(features_pca.columns) + list(reduced.columns) #+ list(cell_type_ohe.columns)

# Want to predict gene expression from SMILES compound formation
X_train = train_samples[xfeatures]
y_train = train_samples.drop(columns=_yfeatures)

X_test = test_samples[xfeatures]
y_test = test_samples.drop(columns=_yfeatures)

## Export

In [95]:
X_train.to_csv("Processed_Data/X_train.csv", index=False)
y_train.to_csv("Processed_Data/y_train.csv", index=False)
X_test.to_csv("Processed_Data/X_test.csv", index=False)
y_test.to_csv("Processed_Data/y_test.csv", index=False)

In [96]:
X_train

,MF PCA 0,MF PCA 1,MF PCA 2,MF PCA 3,MF PCA 4,MF PCA 5,MF PCA 6,MF PCA 7,MF PCA 8,MF PCA 9,...,SC Component 190,SC Component 191,SC Component 192,SC Component 193,SC Component 194,SC Component 195,SC Component 196,SC Component 197,SC Component 198,SC Component 199
0,-0.932676,-0.433296,0.143674,0.431457,-1.911878,0.413157,-0.500679,-0.754016,0.321070,0.096298,...,5.769604,5.749834,2.924859,6.165555,5.486909,4.192760,3.783000,3.587315,4.697591,5.569746
4,-1.225987,0.497097,-1.342844,-0.172098,2.219168,-0.597951,-0.216538,-0.328621,0.883519,0.383401,...,5.769604,5.749834,2.924859,6.165555,5.486909,4.192760,3.783000,3.587315,4.697591,5.569746
10,-0.669821,-0.761423,1.064209,-0.451881,-1.343732,0.269834,0.215291,3.235103,-1.560628,-1.341991,...,5.769604,5.749834,2.924859,6.165555,5.486909,4.192760,3.783000,3.587315,4.697591,5.569746
14,1.610198,0.820041,0.085933,-1.925662,-0.385175,-0.356124,-1.813276,1.588276,-0.374456,0.785268,...,5.769604,5.749834,2.924859,6.165555,5.486909,4.192760,3.783000,3.587315,4.697591,5.569746
18,2.497142,1.749796,0.249717,-1.305727,-1.094619,-0.815843,-1.301174,-0.296460,0.586483,1.555864,...,5.769604,5.749834,2.924859,6.165555,5.486909,4.192760,3.783000,3.587315,4.697591,5.569746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,-0.616064,-0.344641,-0.419898,0.888072,0.741058,-0.711907,0.188368,-2.027478,0.554531,-0.644290,...,5.932283,5.776486,2.907589,6.170081,5.484545,4.201037,3.780656,3.605416,4.674687,5.630428
518,-0.616064,-0.344641,-0.419898,0.888072,0.741058,-0.711907,0.188368,-2.027478,0.554531,-0.644290,...,6.003136,5.825334,2.877749,6.145342,5.468946,4.148154,3.791935,3.538408,4.742606,5.534284
535,-0.082370,-0.992271,-0.945845,0.388957,0.416676,-0.201986,0.701888,0.764968,-0.018647,1.319008,...,5.932283,5.776486,2.907589,6.170081,5.484545,4.201037,3.780656,3.605416,4.674687,5.630428
536,-0.082370,-0.992271,-0.945845,0.388957,0.416676,-0.201986,0.701888,0.764968,-0.018647,1.319008,...,6.003136,5.825334,2.877749,6.145342,5.468946,4.148154,3.791935,3.538408,4.742606,5.534284
